In [1]:
# importing the required libraries
import pandas as pd
import numpy as np
 
# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go

# Forecasting
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
# Coder: Sohel Rana

In [ ]:
df_data_all = pd.read_html('https://docs.google.com/spreadsheets/d/e/2PACX-1vQgQAWwlQYF4XTxVT8sYP5wwqz_KxaWfVNQk9B0FlyPPpDphAIv1cRIMV4ve_1gNbewGjcbkKNpi3Wm/pub?gid=624602850&single=true')
df_data = df_data_all[0].copy()
df_data = df_data.rename(columns={'Unnamed: 2':'District','Unnamed: 3':'Confirmed'})
df_data = df_data[['District', 'Confirmed']]
last_update = df_data['District'][0]
dfd = df_data.drop([0, 1, 67])

print(last_update)
dfd

15-Dec-2020


,District,Confirmed
2,Dhaka City,150629
3,Dhaka (District),6179
4,Gazipur,6694
5,Kishoreganj,3341
6,Madaripur,1599
...,...,...
62,Bogura,9240
63,Natore,1162
64,Naogaon,1499
65,Sirajganj,2489


**Important Links/Api**

In [ ]:
# All Country
http://covid19tracker.gov.bd/api/points

# District in Bangladesh (Confirmed)
http://covid19tracker.gov.bd/api/district

# Dhaka Area (Confirmed)
http://covid19tracker.gov.bd/api/dhaka

# Bangladesh
https://wuhan-coronavirus-api.laeyoung.endpoint.ainize.ai/jhu-edu/timeseries?onlyCountries=true&iso3=BGD
http://covid19tracker.gov.bd/api/country/latest?onlyCountries=true&iso3=USA
http://covid19tracker.gov.bd/

# df.sort_values(by=['Center Name'], inplace=True)

In [4]:
# New Process
iso = input(str('Type iso: '))
df = pd.read_json('https://wuhan-coronavirus-api.laeyoung.endpoint.ainize.ai/jhu-edu/timeseries?onlyCountries=true&iso3='+iso.upper()+'')['timeseries']
df = pd.DataFrame(list(df))
df_new = df.rename(index={0: 'column1'})
df_new = df_new.T # Transpose column
df_new_data = pd.DataFrame(list(df_new['column1']))
df_new_data['Date'] = df_new.index
df_new_data = df_new_data.rename(columns={'confirmed':'Confirmed','deaths':'Deaths','recovered':'Recovered'})
df_new_data['New_Confirmed'] = df_new_data.Confirmed.diff().fillna(0).apply(lambda x : x if x > 0 else 0)
df_new_data['New_Recovered'] = df_new_data.Recovered.diff().fillna(0).apply(lambda x : x if x > 0 else 0)
df_new_data['New_Deaths'] = df_new_data.Deaths.diff().fillna(0).apply(lambda x : x if x > 0 else 0)
df = df_new_data[['Date', 'New_Confirmed', 'Confirmed', 'New_Recovered', 'Recovered', 'New_Deaths', 'Deaths']].query('Confirmed > 0')
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date'], inplace=True)
df.tail()

Type iso: bgd


,Date,New_Confirmed,Confirmed,New_Recovered,Recovered,New_Deaths,Deaths
346,2021-01-02,684.0,515184,964.0,459620,23.0,7599
347,2021-01-03,835.0,516019,978.0,460598,27.0,7626
348,2021-01-04,910.0,516929,917.0,461515,24.0,7650
349,2021-01-05,991.0,517920,944.0,462459,20.0,7670
350,2021-01-06,978.0,518898,1021.0,463480,17.0,7687


In [5]:
# Comparison with Confirmed and Recovered
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y = df['Confirmed'], mode='lines+markers', name='Confirmed',line=dict(color='Red', width=2)))
fig.add_trace(go.Scatter(x=df['Date'], y = df['Recovered'], mode='lines+markers', name='Recovered',line=dict(color='Blue', width=2)))
fig.update_layout(title='Comparision with Confirmed and Recovered', xaxis_tickfont_size=12, yaxis=dict(title='Cases'), plot_bgcolor='rgb(230, 230, 230)', height=600)
fig.show()

In [6]:
# Last 30 days New Test, Recovered and Confirmed
fig = go.Figure(data=[
    go.Bar(x=df["Date"].tail(30), y=df["New_Recovered"].tail(30), name='Recovered', offsetgroup=0),
    go.Bar(x=df["Date"].tail(30), y=df["New_Confirmed"].tail(30), name='Confirmed', offsetgroup=0),
    go.Bar(x=df["Date"].tail(30), y=df["New_Deaths"].tail(30), name='Deaths', offsetgroup=0)
])
fig.update_layout(title_text='Last 14 days New Test, Recovered and Confirmed', xaxis_tickfont_size=12, yaxis=dict(title='Cases'), plot_bgcolor='rgb(230, 230, 230)')
# fig.update_layout(barmode='group')
fig.show()

In [ ]:
Confirmed_days = input('How many days for Confirmed: ')
Confirmed_days = int(Confirmed_days)

Confirmed = df.groupby('Date').sum()['New_Confirmed'].reset_index().tail(Confirmed_days)

How many days for Confirmed: 90


In [ ]:
Confirmed.columns = ['ds','y']
Confirmed['ds'] = pd.to_datetime(Confirmed['ds'])
# Confirmed.tail()

m = Prophet(interval_width=0.95)
m.fit(Confirmed)
future = m.make_future_dataframe(periods=7)
# future.tail()

forecast = m.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

plot_plotly(m, forecast)
# plot_components_plotly(m, forecast) # trend plot

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Another way

In [ ]:
df1 = df[["Date", "New_Confirmed"]]
df1

,Date,New_Confirmed
46,2020-03-08,3.0
47,2020-03-09,0.0
48,2020-03-10,0.0
49,2020-03-11,0.0
50,2020-03-12,0.0
...,...,...
337,2020-12-24,1234.0
338,2020-12-25,1163.0
339,2020-12-26,834.0
340,2020-12-27,1049.0


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# prepare expected column names
df1.columns = ['ds', 'y']
df1['ds']= pd.to_datetime(df1['ds'])

# define the model
model = Prophet()
# fit the model
model.fit(df1)

future = model.make_future_dataframe(7)
# future.tail()

forecast = model.predict(future)
plot_plotly(model, forecast)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
